In [1]:
%cd /Users/alex/PETsARD

/Users/alex/PETsARD


/Users/alex/miniforge3/envs/dev/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
from PETsARD import (
    Loader,
    Processor,
    Synthesizer
)


load = Loader(
    filepath='benchmark://olist'
)
load.load()

Loader - Benchmarker: file benchmark/olist.csv already exist and match SHA-256.
                      PETsARD will ignore download and use local data directly.


In [3]:
synth = Synthesizer(method='sdv-gaussiancopula')

In [4]:
synth.create(load.data, metadata=load.metadata.to_sdv())

Synthesizer (SDV - SingleTable): Metafile loading time: 0.0001 sec.


In [5]:
synth.fit_sample()

Synthesizer (SDV - SingleTable): Fitting GaussianCopula.
Synthesizer (SDV - SingleTable): Fitting  GaussianCopula spent 28.5122 sec.


/Users/alex/miniforge3/envs/dev/lib/python3.10/site-packages/scipy/stats/_continuous_distns.py:700: RuntimeWarning: invalid value encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)


Synthesizer (SDV - SingleTable): Sampling GaussianCopula # 117601 rows (same as raw) in 5.3725 sec.
